In this script we produce the Expert forest map.

We have 5 input maps and they were ranked by some of my colegues from best to worst. 
These input maps are boolean 0/1

We can make a simple (albeit arbitrary) weighted summary.

The idea is rank all 2^5 combinations based on the ranking to be between 0 and 1. 

For this we don't need any training data. 

Implicitly this is just a weighted average, where the weights are 

2^i / (2^5 -1) , where i goes from 0 to 4, with i=4 being the best map. 


the weights are: 0.03225806 0.06451613 0.12903226 0.25806452 0.51612903


In [12]:
%%bash

# A is the worst map
# E the best

gdal_calc.py                           \
--overwrite                            \
-E out_data/JAXA/poland_01.tif         \
-D out_data/C-GLOPS/2019/poland_01.tif \
-A out_data/CLC/poland_01.vrt          \
-C out_data/CLC10m/poland_01.tif       \
-B out_data/GLADGLCextent/reproj3035_100m_tifs/poland_01.tif  \
--outfile=out_data/PL_expert_map_reorder.tif \
--type Float32                    \
--NoDataValue=48                  \
--calc="(A.astype(numpy.float64) *0.03225806  + B.astype(numpy.float64) *0.06451613 + C.astype(numpy.float64) *0.12903226 + D.astype(numpy.float64) *0.25806452 + E.astype(numpy.float64) *0.51612903 )"


0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 

In [8]:
!gdalinfo -hist out_data/expert_map.tif

Driver: GTiff/GeoTIFF
Files: out_data/expert_map.tif
       out_data/expert_map.tif.aux.xml
Size is 7120, 6060
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference Frame 2014"],
            E

In [5]:
%%bash

# Input files
coordinates_file="out_data/LUCAS/LUCAS_PL_XY.csv"

# Output file
output_file="out_data/poland_expert_lucas_extract.csv"

gdallocationinfo out_data/expert_map.tif -geoloc -valonly < $coordinates_file > $output_file



In [7]:
!wc -l out_data/LUCAS/LUCAS_PL_XY.csv
!wc -l out_data/poland_expert_lucas_extract.csv 
!echo --------------------------------
!head out_data/poland_expert_lucas_extract.csv

23084 out_data/LUCAS/LUCAS_PL_XY.csv
23084 out_data/poland_expert_lucas_extract.csv
--------------------------------
0
1
1
0
0
0.822580575942993
0
0
0
1


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
def performance_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = (tp + tn) / (tn + tp + fp + fn)
    tpr = tp / (tp + fn)
    tnr = tn / (tn + fn)
    bacc= (tpr + tnr) * 0.5
#    f1 = (tpr * tnr) / (tpr + tnr)
    return(acc)

In [8]:
outcome = pd.read_csv("out_data/LUCAS/LUCAS_PL_locations.csv", sep=",",  index_col=False)
Y    = outcome.iloc[:,4].to_frame()
print(len(Y))

23084


In [9]:
expert_prop = pd.read_csv("out_data/poland_expert_lucas_extract.csv", sep=",",  index_col=False, header = None, names=['Y_prop'])
print(len(expert_prop))

23084


In [10]:
for thresh in np.linspace(0, 1, num=32):
    pred_temp = expert_prop['Y_prop'] >= thresh
    print( str(round(thresh,4)) + ": " + str(performance_metrics(y_pred = pred_temp,y_true = Y['forest']) ) )

/tmp/ipykernel_2925/2900147056.py:5: RuntimeWarning: invalid value encountered in long_scalars
  tnr = tn / (tn + fn)


0.0: 0.3060128227343615
0.0323: 0.8231675619476694
0.0645: 0.8231675619476694
0.0968: 0.8231675619476694
0.129: 0.8231675619476694
0.1613: 0.8231675619476694
0.1935: 0.8308785305839542
0.2258: 0.8375064980072777
0.2581: 0.8462571478080055
0.2903: 0.8462571478080055
0.3226: 0.8462571478080055
0.3548: 0.8462138277594871
0.3871: 0.8480332697972621
0.4194: 0.8488130306705943
0.4516: 0.8487263905735575
0.4839: 0.8527984751342922
0.5161: 0.8527984751342922
0.5484: 0.85201871426096
0.5806: 0.8514122335817016
0.6129: 0.8510223531450355
0.6452: 0.8488996707676313
0.6774: 0.8488130306705943
0.7097: 0.8488130306705943
0.7419: 0.8488130306705943
0.7742: 0.8470802287298562
0.8065: 0.846300467856524
0.8387: 0.8317882516028418
0.871: 0.8317882516028418
0.9032: 0.8317882516028418
0.9355: 0.8317882516028418
0.9677: 0.8317882516028418
1.0: 0.8317882516028418


alternative ordering

In [11]:
%%bash

# A is the worst map
# E the best

gdal_calc.py                           \
--overwrite                            \
-E out_data/JAXA/poland_01.tif         \
-D out_data/C-GLOPS/2019/poland_01.tif \
-A out_data/CLC/poland_01.vrt          \
-C out_data/CLC10m/poland_01.tif       \
-B out_data/GLADGLCextent/reproj3035_100m_tifs/poland_01.tif  \
--outfile=out_data/expert_map_reorder.tif \
--type Float32                    \
--NoDataValue=48                  \
--calc="(A.astype(numpy.float64) *0.03225806  + B.astype(numpy.float64) *0.06451613 + C.astype(numpy.float64) *0.12903226 + D.astype(numpy.float64) *0.25806452 + E.astype(numpy.float64) *0.51612903 )"


0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 

In [2]:
%%bash

# Input files
coordinates_file="out_data/LUCAS/LUCAS_XY.csv"

# Output file
output_file="out_data/poland_reorderexpert_lucas_extract.csv"

gdallocationinfo out_data/expert_map_reorder.tif -geoloc -valonly < $coordinates_file > $output_file



In [9]:
expertreorder_prop = pd.read_csv("out_data/poland_reorderexpert_lucas_extract.csv", sep=",",  index_col=False, header = None, names=['Y_prop'])
print(len(expertreorder_prop))



23084


In [10]:
for thresh in np.linspace(0, 1, num=32):
    pred_temp = expertreorder_prop['Y_prop'] >= thresh
    print( str(round(thresh,4)) + ": " + str(performance_metrics(y_pred = pred_temp,y_true = Y['forest']) ) )
    

/tmp/ipykernel_9552/2900147056.py:5: RuntimeWarning: invalid value encountered in long_scalars
  tnr = tn / (tn + fn)


0.0: 0.3060128227343615
0.0323: 0.8266331658291457
0.0645: 0.8353838156298735
0.0968: 0.8357303760180211
0.129: 0.8423583434413446
0.1613: 0.8431381043146768
0.1935: 0.8431381043146768
0.2258: 0.8471235487783746
0.2581: 0.8491162710102236
0.2903: 0.8500693120776295
0.3226: 0.8500693120776295
0.3548: 0.8502425922717033
0.3871: 0.8502425922717033
0.4194: 0.849636111592445
0.4516: 0.849636111592445
0.4839: 0.8503292323687403
0.5161: 0.8506757927568879
0.5484: 0.8506757927568879
0.5806: 0.8503725524172587
0.6129: 0.8503725524172587
0.6452: 0.8511089932420725
0.6774: 0.8511089932420725
0.7097: 0.8480332697972621
0.7419: 0.8480332697972621
0.7742: 0.8421850632472708
0.8065: 0.8421850632472708
0.8387: 0.8410154219372725
0.871: 0.8410154219372725
0.9032: 0.8402356610639404
0.9355: 0.8403223011609773
0.9677: 0.8385894992202392
1.0: 0.8317882516028418


In [1]:
%%bash

# A is the worst map
# E the best

start=`date +%s`

gdal_calc.py                           \
--overwrite                            \
-A out_data/JAXA/poland_01.tif         \
-B out_data/C-GLOPS/2019/poland_01.tif \
-C out_data/CLC/poland_01.vrt          \
-D out_data/CLC10m/poland_01.tif       \
-E out_data/GLADGLCextent/reproj3035_100m_tifs/poland_01.tif  \
--outfile=out_data/expert_map.tif \
--type Float32                    \
--NoDataValue=48                  \
--calc="((A.astype(numpy.float64) + B.astype(numpy.float64) + C.astype(numpy.float64) + D.astype(numpy.float64) + E.astype(numpy.float64)+ A.astype(numpy.float64) *0.03225806  + B.astype(numpy.float64) *0.06451613 + C.astype(numpy.float64) *0.12903226 + D.astype(numpy.float64) *0.25806452 + E.astype(numpy.float64) *0.51612903) / 6)"


end=`date +%s`
expr $end - $start



0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 

In [2]:
!gdalinfo -hist out_data/expert_map.tif

Driver: GTiff/GeoTIFF
Files: out_data/expert_map.tif
Size is 7120, 6060
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference Frame 2014"],
            ELLIPSOID["GRS 1980",6378137,298.2572221

In [3]:
%%bash

# A is the worst map
# E the best

start=`date +%s`

gdal_calc.py                           \
--overwrite                            \
-A out_data/JAXA/poland_01.tif         \
-B out_data/C-GLOPS/2019/poland_01.tif \
-C out_data/CLC/poland_01.vrt          \
-D out_data/CLC10m/poland_01.tif       \
-E out_data/GLADGLCextent/reproj3035_100m_tifs/poland_01.tif  \
--outfile=out_data/expert_map.tif \
--type Float32                    \
--NoDataValue=48                  \
--calc="(A.astype(numpy.float64) *0.1720430  + B.astype(numpy.float64) *0.1774194 + C.astype(numpy.float64) *0.1881720 + D.astype(numpy.float64) *0.2096774 + E.astype(numpy.float64) *0.2526882)"


end=`date +%s`
expr $end - $start



0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 

In [1]:
!gdalinfo out_data/JAXA/poland_01.tif 

Driver: GTiff/GeoTIFF
Files: out_data/JAXA/poland_01.tif
Size is 7120, 6060
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference Frame 2014"],
            ELLIPSOID["GRS 1980",6378137,298.257

In [8]:
%%bash

# A is the worst map
# E the best

start=`date +%s`

gdal_calc.py                           \
--overwrite                            \
-A out_data/JAXA/poland_01.tif         \
-B out_data/C-GLOPS/2019/poland_01.tif \
-C out_data/CLC/poland_01.vrt          \
-D out_data/CLC10m/poland_01.tif       \
-E out_data/GLADGLCextent/reproj3035_100m_tifs/poland_01.tif  \
--outfile=out_data/PL_expert_map_integer.tif \
--type Byte                    \
--NoDataValue=255                  \
--calc="(A.astype(numpy.float64) *17.20430  + B.astype(numpy.float64) *17.74194 + C.astype(numpy.float64) *18.81720 + D.astype(numpy.float64) *20.96774 + E.astype(numpy.float64) *25.26882)"


end=`date +%s`
expr $end - $start



0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 

In [9]:
!gdalinfo -hist out_data/PL_expert_map_integer.tif

Driver: GTiff/GeoTIFF
Files: out_data/PL_expert_map_integer.tif
       out_data/PL_expert_map_integer.tif.aux.xml
Size is 7120, 6060
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference Frame